<a href="https://colab.research.google.com/github/Yanh12/MA_automatische_Zusammenfassung/blob/master/abstraktive_Phase/Model2/bilstm_tr_ori_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
from sklearn.model_selection import train_test_split
import os
import io
import numpy as np
import re
import zipfile
import itertools
#import dependencies
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional,LSTMCell
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
from matplotlib import pyplot

2.2.0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


This model is based on TF tuturial (https://www.tensorflow.org/tutorials/text/nmt_with_attention) for NMT using TF 2.0 with modifications.
Specifically to say:
1. modifying how texts are preprocessed; which to be tokenized which not;
2. Using pretrained word2vec instead of the built-in keras embedding;
3. Using BiLstm instead of LSTM for Encoder
4. Other small modifications when needed (eg. some explanations)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/MA_colab')

In [ ]:
#import text files
default_path = '/content/drive/My Drive/MA_colab/bilstm_attn_beam/tr_ori_sum/'
with open ('/content/drive/My Drive/MA_colab/abs_strV2.p', 'rb') as filehandle:
  abs_selected = pickle.load(filehandle)
abs_selected = [x.lower() for x in abs_selected]
with open('/content/drive/My Drive/MA_colab/textRank_texts.p', 'rb') as filehandle:
  texts = pickle.load(filehandle)
texts = [' '.join(x) for x in texts]


In [ ]:
for i in range(5):
    print(texts[i])

after annotating instances based on simple queries and mlsupported ranked queries, we considered the natural next step to be searching automatically for the phenomenon in question utilizing machine learning, using arbitrary sentences from the corpus as input for the classifier instead of only the results returned by a query. our approach is different from active learning as our goal is not to improve the training efficiency of the classifier but rather to allow the user to interactively find and label as many true instances of a phenomenon as possible in a large corpus. consider the following typical scenario incorporating the ranking: a user constructs a query based on various features, such as pos tags or lemmata, which are used to search for matching sentences, e.g. the result is a list of sentences presented in a keywordsincontext view, along with an annotation field . we interpret this as the classifier being successful in helping the annotators after a brief training phase identi

In [ ]:
#tune the text for the further tokenization
to_exclude = '!"#$%&()*+/:;<=>@[\\]^_`{|}~\t\n'
to_tokenize = '.,:;!?-'
texts_modified = []
for article in texts:
  article = re.sub(r'(['+to_tokenize+'])', r' \1 ', article)
  article = re.sub(r'  +', ' ', article)
  texts_modified.append(article)
abs_modified = []
for summary in abs_selected:
  summary = re.sub(r'(['+to_tokenize+'])', r' \1 ', summary)
  summary = re.sub(r'  +', ' ', summary)
  abs_modified.append(summary)

#add START and END tag for each selected abstract
abs_tagged = ['<start> ' + x + ' <end>' for x in abs_modified]
texts_tagged = ['<start> ' + x + ' <end>' for x in texts_modified]

max_len_text = 500
max_len_summary = 100


In [ ]:
#train,validation & test parts
x_tr, x_val, y_tr, y_val = train_test_split(texts_tagged[:15000],abs_tagged[:15000], test_size=0.1,random_state=0,shuffle=False)
x_test = texts_tagged[15000:]
y_test = abs_modified[15000:]

In [ ]:
#text Tokenizer by keras >> convert text sequences into integer representation
x_tokenizer = Tokenizer(filters='')
x_tokenizer.fit_on_texts(list(x_tr))
x_tr = x_tokenizer.texts_to_sequences(x_tr)
x_val = x_tokenizer.texts_to_sequences(x_val)
#padding zero to max length
x_tr = pad_sequences(x_tr,  maxlen=max_len_text, padding='post')
x_val = pad_sequences(x_val, maxlen=max_len_text, padding='post')
#defing the vocabulary size
x_voc_size = len(x_tokenizer.word_index) +1

#summary Tokenizer
y_tokenizer = Tokenizer(filters='')
y_tokenizer.fit_on_texts(list(y_tr))
y_tr = y_tokenizer.texts_to_sequences(y_tr)
y_val = y_tokenizer.texts_to_sequences(y_val)
y_tr = pad_sequences(y_tr, maxlen=max_len_summary, padding='post')
y_val = pad_sequences(y_val, maxlen=max_len_summary, padding='post')
y_voc_size = len(y_tokenizer.word_index) +1

In [ ]:
#word2index
x_word_index = x_tokenizer.word_index
y_word_index = y_tokenizer.word_index

In [ ]:
print(x_tokenizer.word_index['<start>']) 
x_voc_size = len(x_tokenizer.word_index)+1  
y_voc_size = len(y_tokenizer.word_index)+ 1
print("input_vocab_size : ", x_voc_size)
print("output_vocab_size : " ,y_voc_size)

54
input_vocab_size :  136202
output_vocab_size :  32987


In [ ]:
#model parameters
BATCH_SIZE = 32 #due to colab capacity, no bigger batch size possible
BUFFER_SIZE = len(x_tr)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 300
rnn_units = 150
rnn_units_decoder = 300
dense_units = 300
Dtype = tf.float32
Tx = 500 #longst input
Ty = 100 #longst output 

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_tr, y_tr)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
print(example_X.shape) 
print(example_Y.shape)

(32, 500)
(32, 100)


In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
print(example_X.shape) 
print(example_Y.shape)

(32, 500)
(32, 100)


Using Pre-trained embedding models instead of keras Embedding 
Note that there are lots of possible ways to embed words to dense vectors of fixed langth. Keras Embedding modul provides this embedding as other layers-- i.e. minimize loss as its ultimate goal.
However, word2vec tech is the one that trys to capture the SEMANTIC MEANINGS of words, not only simply densing them as minimizing the loss.
Therefore, for text understanding related tasks, better using pre-trained word2vec models as embedding.

In [ ]:

#use pre-trained word embedding as Embedding.layer of the model
#load pretrained word embedding
"""
corpus-specific word embedding can provide some more precise information, but 
it also has the disadvantage that many rare word get ignored.
So, here where use a combined way to better represent the words:
   1. if the word is in the corpus-specific word-embedding file, use it.
   2. if the work is not in this file but in the pretrained embeddings from the
   commonly crawled documents, then use this.
   3. if a word neither in the specific file nor in the common file, initialize
   it with random numbers
"""
cl_embeddings_index = {}
with open('/content/drive/My Drive/MA_colab/CL_word2vec_300dim.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        cl_embeddings_index[word] = coefs


embeddings_index = {}
with open('/content/drive/My Drive/MA_colab/glove.6B.300d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs


#embedding_layer for Encoder
embedding_matrix_encoder = np.zeros((x_voc_size,embedding_dims))
"""
if the word is in the word embedding dict, embedding_vector gets the corresponding value
if not, initilize the vector with random numbers
"""
for word, i in x_word_index.items():
  if word in cl_embeddings_index.keys():
     embedding_vector = cl_embeddings_index.get(word)
     embedding_matrix_encoder[i] = embedding_vector
  elif word in embeddings_index.keys():
     embedding_vector = embeddings_index.get(word)
     embedding_matrix_encoder[i] = embedding_vector
  else:
     embedding_vector=np.random.uniform(-1.0,1.0,(300,))
     embedding_matrix_encoder[i] = embedding_vector


"""
embedding_layer = Embedding ()
initilize the embedding layer:
params: input_dim = the vocabulary size
        output_dim = embedding dimension
        embeddings_initilizer: to initilize the embeddings
        input_length = the max length of the source text
        trainable = False, because we want the pre-trained model and not update it
"""
embedding_layer_encoder = tf.keras.layers.Embedding(input_dim = x_voc_size, 
                                    output_dim = embedding_dims,
                                    weights=[embedding_matrix_encoder], 
                                    input_length = max_len_text, 
                                    trainable = False)

#embedding_layer for Decoder
embedding_matrix_decoder = np.zeros((y_voc_size,embedding_dims))
for word, i in y_word_index.items():
   if word in cl_embeddings_index.keys():
     embedding_vector_y = cl_embeddings_index.get(word)
     embedding_matrix_decoder[i] = embedding_vector_y
   elif word in embeddings_index.keys():
     embedding_vector_y = embeddings_index.get(word)
     embedding_matrix_decoder[i] = embedding_vector_y
   else:
     embedding_vector_y = np.random.uniform(-1.0,1.0,(300,))
     embedding_matrix_decoder[i] = embedding_vector_y

embedding_layer_decoder = tf.keras.layers.Embedding(input_dim = y_voc_size, 
                                    output_dim = embedding_dims,
                                    weights=[embedding_matrix_decoder],
                                    input_length = max_len_summary, 
                                    trainable = False)

In [ ]:
#define the model
#ENCODER
class EncoderNetwork(tf.keras.Model):
    def __init__(self,x_voc_size,embedding_dims, rnn_units):
        super().__init__()
        self.encoder_embedding = embedding_layer_encoder

        #Bi-LSTM
        #not like LSTM, Bi-LSTM has forward and backword hidden state and content state
        #encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
        self.encoder_birnnlayer = Bidirectional(LSTM(rnn_units,return_state=True,return_sequences=True))
        
      
#DECODER
class DecoderNetwork(tf.keras.Model):
    def __init__(self,y_voc_size, embedding_dims, rnn_units_decoder):
        super().__init__()
        self.decoder_embedding = embedding_layer_decoder
        self.dense_layer = Dense(y_voc_size)
        self.decoder_rnncell = LSTMCell(rnn_units_decoder)
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(dense_units,None,BATCH_SIZE*[Tx])
        self.rnn_cell = self.build_rnn_cell(BATCH_SIZE)
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler= self.sampler,
                                                output_layer=self.dense_layer)

    def build_attention_mechanism(self, units,memory, memory_sequence_length):
        return tfa.seq2seq.BahdanauAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)
        #return tfa.seq2seq.LuongAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)

    # wrap decode-rnn cell  
    def build_rnn_cell(self, batch_size ):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism,
                                                attention_layer_size=dense_units)
        return rnn_cell
    
    def build_decoder_initial_state(self, batch_size, encoder_state,Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = batch_size, 
                                                                dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 
        return decoder_initial_state



encoderNetwork = EncoderNetwork(x_voc_size,embedding_dims, rnn_units)
decoderNetwork = DecoderNetwork(y_voc_size,embedding_dims, rnn_units_decoder)

In [ ]:
#optimizing
optimizer = tf.keras.optimizers.Adam()

In [ ]:
#loss function
def loss_function(y_pred, y):
    sparsecategoricalcrossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                                  reduction='none')
    loss = sparsecategoricalcrossentropy(y_true=y, y_pred=y_pred)
    mask = tf.logical_not(tf.math.equal(y,0))  
    mask = tf.cast(mask, dtype=loss.dtype)
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [ ]:
decoderNetwork.attention_mechanism.memory_initialized

False

In [ ]:
#batch training & teacher-forcing
def train_step(input_batch, output_batch, encoder_initial_cell_state):
    #initialize loss = 0
    loss = 0
    with tf.GradientTape() as tape:
        encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)


        #Outputs of BiLstm
        #[last step activations,last memory_state] of encoder passed as input to decoder Network
        encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoderNetwork.encoder_birnnlayer(encoder_emb_inp, 
                                                        initial_state =encoder_initial_cell_state)
        state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
        state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])
       

        
        # Prepare correct Decoder input & output sequence data
        decoder_input = output_batch[:,:-1] # ignore <end>
        #compare logits with timestepped +1 version of decoder_input
        decoder_output = output_batch[:,1:] #ignore <start>


        # Decoder Embeddings
        decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

        #Setting up decoder memory from encoder output and Zero State for AttentionWrapperState
        decoderNetwork.attention_mechanism.setup_memory(encoder_outputs)
        decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                           encoder_state=[state_h, state_c],
                                                                           Dtype=tf.float32)
        
        #BasicDecoderOutput        
        outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                               sequence_length=BATCH_SIZE*[Ty-1])

        logits = outputs.rnn_output
        #Calculate loss
        loss = loss_function(logits, decoder_output)

    #Returns the list of all layer variables / weights.
    variables = encoderNetwork.trainable_variables + decoderNetwork.trainable_variables  
    # differentiate loss wrt variables
    gradients = tape.gradient(loss, variables)

    #grads_and_vars – List of(gradient, variable) pairs.
    grads_and_vars = zip(gradients,variables)
    optimizer.apply_gradients(grads_and_vars)
    return loss

In [ ]:
#training
chkpoint_prefix = os.path.join(default_path, "chkpoint")
if not os.path.exists(default_path):
    os.mkdir(default_path)

checkpoint = tf.train.Checkpoint(optimizer = optimizer, encoderNetwork = encoderNetwork, 
                                 decoderNetwork = decoderNetwork)

try:
    status = checkpoint.restore(tf.train.latest_checkpoint(default_path))
    print("Checkpoint found at {}".format(tf.train.latest_checkpoint(default_path)))
except:
    print("No checkpoint found at {}".format(default_path))


Checkpoint found at /content/drive/My Drive/MA_colab/bilstm_attn_beam/tr_ori_sum/chkpoint-68


In [ ]:
#RNN LSTM hidden and memory state initializer
def initialize_initial_state():
    #for Bi-LSTM
    return [tf.zeros((BATCH_SIZE, rnn_units)) for i in range(4)]
   

In [ ]:
epochs = 10
loss=[]
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0
    #print(encoder_initial_cell_state)

    
    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%100 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))
            checkpoint.save(file_prefix = chkpoint_prefix)
            loss.append(batch_loss.numpy())
    

total loss: 3.3351662158966064 epoch 1 batch 100 
total loss: 3.5954418182373047 epoch 1 batch 200 
total loss: 3.341259241104126 epoch 1 batch 300 
total loss: 3.433643341064453 epoch 1 batch 400 
total loss: 3.1968581676483154 epoch 2 batch 100 
total loss: 3.146852731704712 epoch 2 batch 200 
total loss: 3.09846830368042 epoch 2 batch 300 
total loss: 3.502392292022705 epoch 2 batch 400 
total loss: 3.058649778366089 epoch 3 batch 100 
total loss: 3.551872730255127 epoch 3 batch 200 
total loss: 3.401801586151123 epoch 3 batch 300 
total loss: 3.3741252422332764 epoch 3 batch 400 
total loss: 3.292229175567627 epoch 4 batch 100 
total loss: 3.2687160968780518 epoch 4 batch 200 
total loss: 3.1418309211730957 epoch 4 batch 300 
total loss: 3.1170547008514404 epoch 4 batch 400 
total loss: 2.932401180267334 epoch 5 batch 100 
total loss: 2.820590019226074 epoch 5 batch 200 
total loss: 2.9926443099975586 epoch 5 batch 300 
total loss: 3.1964428424835205 epoch 5 batch 400 
total loss: 

In [ ]:
#validation
epochs = 10
loss_val=[]
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0
    #print(encoder_initial_cell_state)

    
    for ( batch , (input_batch, output_batch)) in enumerate(dataset_test.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%10 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))
            checkpoint.save(file_prefix = chkpoint_prefix)
            loss_val.append(batch_loss.numpy())

total loss: 3.7752366065979004 epoch 1 batch 10 
total loss: 3.9857006072998047 epoch 1 batch 20 
total loss: 3.6837124824523926 epoch 1 batch 30 
total loss: 3.915937662124634 epoch 1 batch 40 
total loss: 3.509152412414551 epoch 2 batch 10 
total loss: 3.3640096187591553 epoch 2 batch 20 
total loss: 3.4372410774230957 epoch 2 batch 30 
total loss: 3.402860403060913 epoch 2 batch 40 
total loss: 2.954256534576416 epoch 3 batch 10 
total loss: 2.8961548805236816 epoch 3 batch 20 
total loss: 3.1395833492279053 epoch 3 batch 30 
total loss: 3.0394182205200195 epoch 3 batch 40 
total loss: 2.8519339561462402 epoch 4 batch 10 
total loss: 2.6021552085876465 epoch 4 batch 20 
total loss: 2.731849431991577 epoch 4 batch 30 
total loss: 2.9121248722076416 epoch 4 batch 40 
total loss: 2.3856875896453857 epoch 5 batch 10 
total loss: 2.670823812484741 epoch 5 batch 20 
total loss: 2.7404394149780273 epoch 5 batch 30 
total loss: 2.4590656757354736 epoch 5 batch 40 
total loss: 2.300996303558

In [ ]:
#inference
#if trained in same session else use checkpoint variable
#decoder_embedding_matrix = tf.train.load_variable(checkpointdir, 'decoderNetwork/decoder_embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE')
decoder_embedding_matrix = decoderNetwork.decoder_embedding.variables[0] 
print(decoderNetwork.decoder_embedding.variables[0].shape)

(32987, 300)


In [ ]:
[print(var) for var in tf.train.list_variables(
    default_path) if re.match(r'.*decoder_embedding.*',var[0])]

('decoderNetwork/decoder_embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE', [32987, 300])


[None]

In [ ]:
decoder_embedding_matrix = tf.train.load_variable(
    default_path, 'decoderNetwork/decoder_embedding/embeddings/.ATTRIBUTES/VARIABLE_VALUE')
print(decoder_embedding_matrix.shape)

(32987, 300)


In [ ]:
x_test=texts_tagged[15000:]
x_test = x_tokenizer.texts_to_sequences(x_test)
input_x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len_text, padding='post')


In [ ]:
end_token = y_tokenizer.word_index['<end>']

In [ ]:
def test (test_input):

    beam_width = 8

    inference_batch_size = test_input.shape[0]
    inp = tf.convert_to_tensor(test_input)            
    encoder_emb_inp = encoderNetwork.encoder_embedding(inp)

    def initialize_initial_state_inference():
        #for Bi-LSTM
        return [tf.zeros((inference_batch_size, rnn_units)) for i in range(4)]

    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoderNetwork.encoder_birnnlayer(encoder_emb_inp, 
                                                        initial_state =initialize_initial_state_inference())
    state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
    state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])

    start_tokens = tf.fill([inference_batch_size],y_tokenizer.word_index['<start>'])
    end_token = y_tokenizer.word_index['<end>']

    decoder_input = tf.expand_dims([y_tokenizer.word_index['<start>']]* inference_batch_size,1)
    decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

    encoder_memory = tfa.seq2seq.tile_batch(encoder_outputs, beam_width)
    decoderNetwork.attention_mechanism.setup_memory(encoder_memory)
    print("beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] :", encoder_memory.shape)
    #set decoder_inital_state which is an AttentionWrapperState considering beam_width
    decoder_initial_state = decoderNetwork.rnn_cell.get_initial_state(batch_size = inference_batch_size* beam_width,dtype = Dtype)
    encoder_state = tfa.seq2seq.tile_batch([state_h, state_c], multiplier=beam_width)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 

    decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoderNetwork.rnn_cell,beam_width=beam_width,
                                                 output_layer=decoderNetwork.dense_layer)


    # Since we do not know the target sequence lengths in advance, we use maximum_iterations to limit the translation lengths.
    # One heuristic is to decode up to two times the source sentence lengths.
    maximum_iterations = tf.round(tf.reduce_max(Tx) * 2)

    #initialize inference decoder

    (first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                                start_tokens = start_tokens,
                                end_token=end_token,
                                initial_state = decoder_initial_state)
    #print( first_finished.shape)
    #print("\nfirst_inputs returns the same decoder_input i.e. embedding of  <start> :",first_inputs.shape)

    inputs = first_inputs
    state = first_state  
    predictions = np.empty((inference_batch_size, beam_width,0), dtype = np.int32)
    beam_scores =  np.empty((inference_batch_size, beam_width,0), dtype = np.float32)                                                                            
    for j in range(maximum_iterations):
        beam_search_outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
        inputs = next_inputs
        state = next_state
        outputs = np.expand_dims(beam_search_outputs.predicted_ids,axis = -1)
        scores = np.expand_dims(beam_search_outputs.scores,axis = -1)
        predictions = np.append(predictions, outputs, axis = -1)
        beam_scores = np.append(beam_scores, scores, axis = -1)
    #print(predictions.shape) 
    #print(beam_scores.shape)
    
    return predictions, beam_scores

In [ ]:
#due to the limitattion of gpu memory, test is done in batchs
test_batch = 32
iters = len(x_test) // test_batch 
last_iter = iters+1
all_predictions = []
all_beam_scores = []
for i in range(iters+1):
    test_input = input_x_test[test_batch*i : test_batch*(i+1)]
    predictions, beam_scores = test(test_input)
    all_predictions.append(predictions)
    all_beam_scores.append(beam_scores)
    if i == last_iter:
        test_input = input_x_test[test_batch*last_iter:]
        predictions, beam_scores = test(test_input)
        all_predictions.append(predictions)
        all_beam_scores.append(beam_scores)


beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (256, 500, 300)
beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]

In [ ]:
predicted_sum = []
for i in range (len(all_predictions)):
    #select the predicted summary with highest beam score as the final result
    predictions = all_predictions[i]
    beam_scores = all_beam_scores[i]
    for j in range(len(predictions)):
        output_beams_per_sample = predictions[j,:,:]
        score_beams_per_sample = beam_scores[j,:,:]
        for beam, score in zip(output_beams_per_sample,score_beams_per_sample) :
            seq = list(itertools.takewhile( lambda index: index !=end_token, beam))
            score_indexes = np.arange(len(seq))
            beam_score = score[score_indexes].sum()
            predicted = sorted ({(" ".join( [y_tokenizer.index_word[w] for w in seq])):beam_score}.keys(),reverse=True)[0]
        predicted_sum.append(predicted)

#store the results
for i in range(len(predicted_sum)):
    ref = open(default_path + "ref/ref_{}.txt".format(i), 'w')
    ref.write(y_test[i])
    ref.close()
    summ = open(default_path + "predicted/sum_{}.txt".format(i), 'w')
    summ.write(predicted_sum[i])
    summ.close()
  


In [ ]:
#to see beam scores of a summary
for i in range(len(predicted_sum[:5])):
    print("-----------------")
    print("Original Summary:")
    print(y_test[i])
    #print("-----------------")
    #print("\nPredicted Summary:")
    #print("---------------------------------------------")
    output_beams_per_sample = predictions[i,:,:]
    score_beams_per_sample = beam_scores[i,:,:]
    for j,(beam, score) in enumerate(zip(output_beams_per_sample,score_beams_per_sample)) :
        seq = list(itertools.takewhile( lambda index: index !=end_token, beam))
        score_indexes = np.arange(len(seq))
        beam_score = score[score_indexes].sum()
        #print ("version{}: ".format(j), (" ".join( [y_tokenizer.index_word[w] for w in seq]), " beam score: ", beam_score))
        #print (" ".join( [y_tokenizer.index_word[w] for w in seq]), " beam score: ", beam_score)
        predicted = sorted ({(" ".join( [y_tokenizer.index_word[w] for w in seq])):beam_score}.keys(),reverse=True)[0]
    print("---------------------------------------------")
    print("best summary: " + predicted)
    print('/n')
  

-----------------
Original Summary:
 despite the popularity of stochastic parsers , symbolic parsing still has some advantages , but is not practical without an effective mechanism for selecting among alternative analyses . this paper describes the symbolic preference system of a hybrid parser that combines a shallow parser with an overlay parser that builds on the chunks . the hybrid currently equals or exceeds most stochastic parsers in speed and is approaching them in accuracy . the preference system is novel in using a simple , threevalued scoring method for assigning preferences to constituents viewed in the context of their containing constituents . the approach addresses problems associated with earlier preference systems , and has considerably facilitated development . it is ultimately based on viewing preference scoring as an engineering mechanism , and only indirectly related to cognitive principles or corpusbased frequencies . 
---------------------------------------------
b